## **EX3 - Transitioning from PFMs to PPMs to PWMs**

* This exercise is to be solved strictly on-site
* Requirements
  * gata3.txt file from L3-1
  * First block of code from the relevant exercise in L3-1 notebook (we have copied it below for your convenience)
* What are PFMs, PPMs and PWMs?
  * They are frequently used to describe the sequence motif preference of transcription factors (TFs)
  * TFs are DNA binding proteins that regulate transcription throughput
  * When such sequence motifs are encountered on genomic sequences, there's a higher probability this particular location to be bound by the TF that recognizes this specific motif
  * PFMs provide the frequency of each letter of the alphabet {A, T, G, C} at each position of the motif
  * PPMs are calculated from PFMs and they provide the probability of encountering each letter of the alphabet at each position of the motif
  * PWMs are calculated from PPMs and they provide a normalized score for each letter of the alphabet at each position of the motif
* Common practices
  * We usually add pseudocounts to the PFM values before transitioning to a PPM
  * For keeping things simple, in this exercise we will just add 1 to every value in the PFM
  * To calculate the PWM, we need background probabilities of each alphabet letter at each position
  * To keep things simple, we will use 0.25 for all letters at each position
* "EX3- Help on Generating_PWMs.pptx" file in the teams folder includes a detailed example of how to generate a PWM

**Loading the sequences from gata3.txt**

* First block of code from the relevant exercise in L3-1 notebook
* Feel free to adjust the code as you see fit

In [1]:
sequences_file = 'gata3.txt'

sequences_as_list_of_lists = list() # initiate an empty list to hold the information that we will load from the file
seq_file_fh = open(sequences_file, 'r') # open a connection to the file and attach it to a file handle
for line in seq_file_fh: # use the file handle as a proxy to loop through the lines of the file one-by-one
    line = line.rstrip('\n') # remove the newline character from the end of the string
    #print(line)
    
    line_as_list = list(line.upper())
    #print(line_as_list)
    
    sequences_as_list_of_lists.append(line_as_list)
    #print(sequences_as_list_of_lists)
seq_file_fh.close() # detach the file handle from the file and close the connection to the file

print(sequences_as_list_of_lists)

[['A', 'G', 'A', 'T', 'A', 'G'], ['A', 'G', 'A', 'T', 'A', 'G'], ['A', 'G', 'A', 'T', 'A', 'G'], ['A', 'G', 'A', 'T', 'A', 'G'], ['C', 'G', 'A', 'T', 'A', 'G'], ['A', 'G', 'A', 'T', 'A', 'C'], ['A', 'G', 'A', 'T', 'A', 'A'], ['G', 'G', 'A', 'T', 'A', 'C'], ['C', 'G', 'A', 'T', 'T', 'G'], ['C', 'G', 'A', 'T', 'T', 'G'], ['A', 'G', 'A', 'T', 'A', 'G'], ['A', 'G', 'A', 'G', 'A', 'G'], ['C', 'G', 'A', 'T', 'A', 'C'], ['A', 'G', 'A', 'T', 'T', 'G'], ['C', 'G', 'A', 'T', 'A', 'G'], ['A', 'G', 'A', 'T', 'A', 'C'], ['A', 'G', 'A', 'T', 'A', 'A'], ['G', 'G', 'A', 'T', 'A', 'C'], ['C', 'G', 'A', 'T', 'T', 'G'], ['C', 'G', 'A', 'T', 'T', 'G']]


**Generate the PFM given the content of sequences_as_list_of_lists**

* You can use a 2-d list or... a more versatile data structure
* Don't forget to add the pseudocounts

In [2]:
with open('gata3.txt', 'r') as file:
    sequences = file.readlines()

new = ['']*(len(sequences))
for index, line in enumerate(sequences):
    new[index] = line.upper().strip()


def matrix(array) -> dict:
    """Returns a dictionary with the position frequency matrix of a list of sequences"""
    # Creating a data structure to help formatting
    # Getting the length of the sequences
    positions = len(array[0])
    # Making a list for each nucleotide that contains the amount in the positions that relates to the index of the list
    indecies = {"A": [1]*positions, "T": [1]*positions, "G": [1]*positions, "C": [1]*positions} # all locations have 1 for the pseudocounts
    # Iterating sequences (plural)
    for seq in array:
        # Iterating sequence (singular)
        for position, nuc in enumerate(seq):
            indecies[nuc][position] += 1
    return indecies

result = matrix(new)
print(result)


{'A': [12, 1, 21, 1, 16, 3], 'T': [1, 1, 1, 20, 6, 1], 'G': [3, 21, 1, 2, 1, 14], 'C': [8, 1, 1, 1, 1, 6]}


**Generate the PPM given the previously calculated PFM**

* **Reminder:** To calculate the probability of each letter at each position, simply divide the frequency at each position with the number of sequences we have in total

In [3]:
for key in result:
    positions = result[key]
    output = {"A": [0]*(len(positions)), "T": [0]*(len(positions)), "G": [0]*(len(positions)), "C": [0]*(len(positions))}
    for i in range(len(positions)):
        adenine = result['A'][i]
        thymine = result['T'][i]
        guanine = result['G'][i]
        cytosine = result['C'][i]
        sum = adenine + thymine + guanine + cytosine
        aw = adenine/sum
        tw = thymine/sum
        gw = guanine/sum
        cw = cytosine/sum
        output['A'][i] = aw
        output['T'][i] = tw
        output['G'][i] = gw
        output['C'][i] = cw
print(output)

{'A': [0.5, 0.041666666666666664, 0.875, 0.041666666666666664, 0.6666666666666666, 0.125], 'T': [0.041666666666666664, 0.041666666666666664, 0.041666666666666664, 0.8333333333333334, 0.25, 0.041666666666666664], 'G': [0.125, 0.875, 0.041666666666666664, 0.08333333333333333, 0.041666666666666664, 0.5833333333333334], 'C': [0.3333333333333333, 0.041666666666666664, 0.041666666666666664, 0.041666666666666664, 0.041666666666666664, 0.25]}


**Generate the PWM given the previously calculated PPM**

* **Reminder:** To calculate the score of each letter at each position, simply divide the probability at each position with the corresponding background probability and then pass the results through the log2 function (import math)
* PWM(i,j) = log2(PPM(i,j) / bkgr(i,j))

In [4]:
import math

background = 0.25

for key in output:
    weight = output[key]
    final =  {"A": [0]*(len(positions)), "T": [0]*(len(positions)), "G": [0]*(len(positions)), "C": [0]*(len(positions))}
    for i in range(len(weight)):
        adenine = output['A'][i]
        thymine = output['T'][i]
        guanine = output['G'][i]
        cytosine = output['C'][i]
        aw = math.log2(adenine/background)
        tw = math.log2(thymine/background)
        gw = math.log2(guanine/background)
        cw = math.log2(cytosine/background)
        final['A'][i] = aw
        final['T'][i] = tw
        final['G'][i] = gw
        final['C'][i] = cw
print(final)

{'A': [1.0, -2.584962500721156, 1.8073549220576042, -2.584962500721156, 1.4150374992788437, -1.0], 'T': [-2.584962500721156, -2.584962500721156, -2.584962500721156, 1.7369655941662063, 0.0, -2.584962500721156], 'G': [-1.0, 1.8073549220576042, -2.584962500721156, -1.5849625007211563, -2.584962500721156, 1.222392421336448], 'C': [0.41503749927884376, -2.584962500721156, -2.584962500721156, -2.584962500721156, -2.584962500721156, 0.0]}
